In [16]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))
sys.setrecursionlimit(20500)
import pandas as pd
import spacy

import matplotlib.pyplot as plt
import clean_dataset as clean
import make_dataset as mk
import vectorize_embed as em
import tools as tools
import models as m

import pickle
import numpy as np

'''features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import Pipeline

'''Metrics/Evaluation'''
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

import joblib
import seaborn as sns

In [42]:
'''import data'''
df = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed/'))+'/taxonomy_final.csv')  
df['all_text_clean_spacy'] = df['all_text_clean_spacy'].astype(str).apply(clean.basic)
df['all_text_clean'] = df['all_text_clean'].astype(str)

In [47]:
cats = df.drop(columns=['PIMS_ID','all_text', 'all_text_clean', 'all_text_clean_spacy', 'hyperlink', 'title',
                       'leading_country', 'grant_amount', 'country_code', 'lon', 'lat'])
categories = cats.columns.tolist()

with open(os.path.abspath(os.path.join('..', 'data/processed/'))+"/all_categories_list.pkl", 'wb') as handle:
    pickle.dump(categories, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
categories_used = []
scores = []

## Tf-idf only vectorization

In [44]:
# iterate over categories and save best models:
    #Creating a dict of the models
model_dict = {
              'LR_balanced' : LogisticRegression(random_state = 3, class_weight = "balanced"),
              'LR' : LogisticRegression(random_state = 3),
              'SDG' : SGDClassifier(random_state=3, loss='log'),
              'SDG_balanceed' : SGDClassifier(random_state=3, loss='log',class_weight = "balanced" ),
              'RF': RandomForestClassifier(random_state=3),
              #'Decsision_Tree': DecisionTreeClassifier(random_state=3),
              #'AdaBoost': AdaBoostClassifier(random_state=3),
              #'KNB': KNeighborsClassifier()
                }

for category in categories: 
    if category != "no tag":
        if df[category].sum(axis=0) > 20:
            
            print('processing:', category)
            print('_____')
            
            X_train, X_test, y_train, y_test = train_test_split(df['all_text_clean_spacy'],                                                    
                                                    df[category],
                                                    test_size = .3,
                                                    random_state = 1,
                                                    stratify = df[category].values, 
                                                    shuffle = True)

            tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                               min_df = 2, 
                                               max_df = .95)
            
            vect = tfidf_vectorizer.fit(X_train)
            X_train = vect.transform(X_train).toarray()
            X_test = vect.transform(X_test).toarray()

            if len(category) > 50:
                 shorter = category[0:20]
                 #save vectorizer:
                 filename = '../models/tf_idf/tf_idf_only/'+shorter+'_'+'vectorizer.sav'
                 joblib.dump(vect, filename)    

            else:
                #save vectorizer:
                filename = '../models/tf_idf/tf_idf_only/'+category+'_'+'vectorizer.sav'
                joblib.dump(vect, filename)    


            results, score = m.model_score_df_all(model_dict, category, 'tf_idf_only', X_train, X_test, y_train, y_test)
            
            print(score)
            print(category)
            
            categories_used.append(category)
            scores.append(score)
            print(scores)
            print(categories_used)
            
info_dict = dict(zip(categories_used, scores))

with open(os.path.abspath(os.path.join('..', 'data/processed/'))+"/tfidf_only_f1.pkl", 'wb') as handle:
    pickle.dump(info_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

processing: forests
_____
0.7829795775001254
forests
[0.7829795775001254]
['forests']
processing: conserve_areas
_____
0.831219512195122
conserve_areas
[0.7829795775001254, 0.831219512195122]
['forests', 'conserve_areas']
processing: freshwaters
_____
0.6424453388447733
freshwaters
[0.7829795775001254, 0.831219512195122, 0.6424453388447733]
['forests', 'conserve_areas', 'freshwaters']
processing: grasslands
_____
0.8057053009883199
grasslands
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199]
['forests', 'conserve_areas', 'freshwaters', 'grasslands']
processing: marine
_____
0.8454614168899883
marine
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883]
['forests', 'conserve_areas', 'freshwaters', 'grasslands', 'marine']
processing: wetlands
_____
0.6483739837398373
wetlands
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373]
['forests', 'conser

0.6197802197802198
tropical_forests
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198]
['forests', 'conserve_areas', 'freshwaters', 'grasslands', 'marine', 'wetlands', 'human_altered_areas', 'coasts', 'coral_reefs', 'dryland_forests', 'indigenous_and_communities_conserved_areas_iccas', 'industrial_site', 'key_biodiversity_areas_kbas', 'marine_and_coastal_protected_areas', 'montane_forests', 'productive_landscapes_seascapes', 'rivers_and_river_basins', 'rural_areas', 'terrestrial_protected_areas', 'tropical_forests']
processing: urban_areas
_____
0.790415937542065
urban_areas
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.805705300

0.9537259936874766
energy_program_of_climate_change_mitigation
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198, 0.790415937542065, 0.7560926485397784, 0.9767128819491183, 0.9329023917259212, 0.9651560926485399, 0.9900488927236124, 0.9458867688457929, 0.9537259936874766]
['forests', 'conserve_areas', 'freshwaters', 'grasslands', 'marine', 'wetlands', 'human_altered_areas', 'coasts', 'coral_reefs', 'dryland_forests', 'indigenous_and_communities_conserved_areas_iccas', 'industrial_site', 'key_biodiversity_areas_kbas', 'marine_and_coastal_protected_areas', 'montane_forests', 'productive_landscapes_seascapes', 'rivers_and_river_basins', 'rural_area

0.8940785445639814
energy_efficiency
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198, 0.790415937542065, 0.7560926485397784, 0.9767128819491183, 0.9329023917259212, 0.9651560926485399, 0.9900488927236124, 0.9458867688457929, 0.9537259936874766, 0.9239560439560439, 0.8543771043771045, 0.790415937542065, 0.8290851610353686, 0.8306320026106822, 0.8940785445639814]
['forests', 'conserve_areas', 'freshwaters', 'grasslands', 'marine', 'wetlands', 'human_altered_areas', 'coasts', 'coral_reefs', 'dryland_forests', 'indigenous_and_communities_conserved_areas_iccas', 'industrial_site', 'key_biodiversity_areas_kbas', 'marine_and_coastal_protected_areas',

0.851118760757315
persistent_organic_pollutants
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198, 0.790415937542065, 0.7560926485397784, 0.9767128819491183, 0.9329023917259212, 0.9651560926485399, 0.9900488927236124, 0.9458867688457929, 0.9537259936874766, 0.9239560439560439, 0.8543771043771045, 0.790415937542065, 0.8290851610353686, 0.8306320026106822, 0.8940785445639814, 0.7069783197831978, 0.7501514092709062, 0.8947688564476886, 0.7022375215146299, 0.851118760757315]
['forests', 'conserve_areas', 'freshwaters', 'grasslands', 'marine', 'wetlands', 'human_altered_areas', 'coasts', 'coral_reefs', 'dryland_forests', 'indigenous_and_communities_c

...

0.7449753712292813
private_sector
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198, 0.790415937542065, 0.7560926485397784, 0.9767128819491183, 0.9329023917259212, 0.9651560926485399, 0.9900488927236124, 0.9458867688457929, 0.9537259936874766, 0.9239560439560439, 0.8543771043771045, 0.790415937542065, 0.8290851610353686, 0.8306320026106822, 0.8940785445639814, 0.7069783197831978, 0.7501514092709062, 0.8947688564476886, 0.7022375215146299, 0.851118760757315, 0.9031587365053979, 0.8109289617486339, 0.8257804632426988, 0.8273453093812375, 0.7194056401402187, 0.7652645861601085, 0.6727027027027028, 0.8607757926927411, 0.7561090225563909, 0.758453226

0.7120349939999226
women
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198, 0.790415937542065, 0.7560926485397784, 0.9767128819491183, 0.9329023917259212, 0.9651560926485399, 0.9900488927236124, 0.9458867688457929, 0.9537259936874766, 0.9239560439560439, 0.8543771043771045, 0.790415937542065, 0.8290851610353686, 0.8306320026106822, 0.8940785445639814, 0.7069783197831978, 0.7501514092709062, 0.8947688564476886, 0.7022375215146299, 0.851118760757315, 0.9031587365053979, 0.8109289617486339, 0.8257804632426988, 0.8273453093812375, 0.7194056401402187, 0.7652645861601085, 0.6727027027027028, 0.8607757926927411, 0.7561090225563909, 0.7584532264918351, 

0.8597297297297297
capital_providers
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198, 0.790415937542065, 0.7560926485397784, 0.9767128819491183, 0.9329023917259212, 0.9651560926485399, 0.9900488927236124, 0.9458867688457929, 0.9537259936874766, 0.9239560439560439, 0.8543771043771045, 0.790415937542065, 0.8290851610353686, 0.8306320026106822, 0.8940785445639814, 0.7069783197831978, 0.7501514092709062, 0.8947688564476886, 0.7022375215146299, 0.851118760757315, 0.9031587365053979, 0.8109289617486339, 0.8257804632426988, 0.8273453093812375, 0.7194056401402187, 0.7652645861601085, 0.6727027027027028, 0.8607757926927411, 0.7561090225563909, 0.758453

0.6613534675615212
individuals_entrepreneurs
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198, 0.790415937542065, 0.7560926485397784, 0.9767128819491183, 0.9329023917259212, 0.9651560926485399, 0.9900488927236124, 0.9458867688457929, 0.9537259936874766, 0.9239560439560439, 0.8543771043771045, 0.790415937542065, 0.8290851610353686, 0.8306320026106822, 0.8940785445639814, 0.7069783197831978, 0.7501514092709062, 0.8947688564476886, 0.7022375215146299, 0.851118760757315, 0.9031587365053979, 0.8109289617486339, 0.8257804632426988, 0.8273453093812375, 0.7194056401402187, 0.7652645861601085, 0.6727027027027028, 0.8607757926927411, 0.7561090225563909, 

0.637807881773399
small_and_medium_sized_enterprises
[0.7829795775001254, 0.831219512195122, 0.6424453388447733, 0.8057053009883199, 0.8454614168899883, 0.6483739837398373, 0.7631384939441812, 0.7487656113854197, 0.7652645861601085, 0.8288822947576656, 0.671370420624152, 0.8746376811594203, 0.8032887256188227, 0.781012658227848, 0.6007253544345532, 0.7177200043511367, 0.8050861097698374, 0.7743478260869565, 0.7832080200501252, 0.6197802197802198, 0.790415937542065, 0.7560926485397784, 0.9767128819491183, 0.9329023917259212, 0.9651560926485399, 0.9900488927236124, 0.9458867688457929, 0.9537259936874766, 0.9239560439560439, 0.8543771043771045, 0.790415937542065, 0.8290851610353686, 0.8306320026106822, 0.8940785445639814, 0.7069783197831978, 0.7501514092709062, 0.8947688564476886, 0.7022375215146299, 0.851118760757315, 0.9031587365053979, 0.8109289617486339, 0.8257804632426988, 0.8273453093812375, 0.7194056401402187, 0.7652645861601085, 0.6727027027027028, 0.8607757926927411, 0.7561090225